<a href="https://colab.research.google.com/github/BhuvanKalyanGV/Web_scapping/blob/main/Futures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium

In [ ]:
!pip install undetected-chromedriver

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Send a GET request to the website
url = "https://www.eex.com/en/market-data/natural-gas/futures"
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract gas types from the dropdown menu
gas_types = []
select_tag = soup.find('select', {'class': 'selectpicker'})
for option in select_tag.find_all('option'):
    gas_types.append({'name': option.text, 'value': option['value']})

# Display the list of gas types to the user
print("Select a gas type by entering the corresponding integer value:")
for i, gas_type in enumerate(gas_types):
    print(f"{i+1}. {gas_type['name']} - Value: {gas_type['value']}")

# Get user input for the gas type value
gas_type_value = int(input("Enter the gas type value: "))

# Load the URL with the selected gas type value
selected_gas_type = next(gas_type for gas_type in gas_types if gas_type['value'] == str(gas_type_value))
url_with_gas_type = f"https://www.eex.com/en/market-data/natural-gas/futures#%7B%22snippetpicker%22%3A%22{gas_type_value}%22%7D"
response = requests.get(url_with_gas_type)
soup = BeautifulSoup(response.content, 'html.parser')

# Extract categories from the div tags with class name "mv-button-base mv-hyperlink-button"
categories = []
for div_tag in soup.find_all('div', {'class': 'mv-button-base mv-hyperlink-button'}):
    categories.append(div_tag.text)

# Create an Excel file and scrape data for each category
with pd.ExcelWriter(f"{selected_gas_type['name']}.xlsx") as writer:
    # Create a new sheet in the Excel file
    pd.DataFrame().to_excel(writer, sheet_name="Sheet1", index=False)

    for category in categories:
        # Extract table data for the current category
        table_data = []
        table_tag = soup.find('table', {'data-category': category})
        for row in table_tag.find_all('tr'):
            row_data = [cell.text for cell in row.find_all('td')]
            table_data.append(row_data)

        # Create a DataFrame from the table data
        df = pd.DataFrame(table_data[1:], columns=table_data[0])

        # Highlight the column names
        df.style.set_properties(subset=pd.IndexSlice[0, :], **{'background-color': 'yellow'})

        # Write the DataFrame to a separate sheet in the Excel file
        df.to_excel(writer, sheet_name=category, index=False)

print(f"Data scraped and saved to {selected_gas_type['name']}.xlsx")